In [106]:
# Import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as sp_int
import scipy.sparse as sp_sp
import scipy.sparse.linalg as sp_la
import scipy.signal as sp_sgn
import scipy.fft as sp_fft
pi=np.pi
# For graphic display in Jupyter Notebook
%matplotlib qt

In [107]:
# Defining classes: here I made 3 classes, which are: Gaussian, Kernel, Kernel_Turing_model.
# The three have a hierarchy of Gaussian < Kernel < Kernel_Turing_model,


#
# class Gaussian defines the Gaussian function needed to define the activator (A(x))
# and inhibitor (I(x)). 
#
# Parameters:
#
# Gaussian.amp (float) ... the amplitude of the Gaussian function (value of the peak)
#
# Gaussian.width (float) ... this was unclear in the original paper, as "width" of a Gaussian distribution is infinite...
#                            here I assume that it is supposed to be the standard deviation which defines how "fat" the function will be.
#
# Gaussian.dist (float) ... the mean of the Gaussian function, in other words the distance from the origin.
#
#
# Methods:
#
# Gaussian.func(x) ... returns the function values of the Gaussian
# 
#   Input: x - a numpy array of any dimension.
#
#   Output: the Gaussian function values at x.
#
# (To be honest this class was not really necessary... but it makes nice organizastion!)
#

class Gaussian():
  def __init__(self, _amp, _width, _dist):
    self.amp = _amp
    self.width = _width
    self.dist = _dist

  def func(self, x):
    return self.amp/(2*pi)**0.5 * np.exp(-((x-self.dist)/self.width)**2/2)

#
# class Kernel is the kernel used for the convolution to calculate the stimulus.
# It is defined with two Gaussian classes, one activator (positive) and one inhibitor (negative).
# It is initialized from the (amp, width, dist) x 2 of the two Gaussian classes.
# It has methods for calculating its integral value at [-inf, inf], performing Fourier transform, and visualization.
#
# Parameters:
#
# Kernel.A (class Gaussian) ... the activator Gaussian function A(x)
#
# Kernel.I (class Gaussian) ... the inhibitor Gaussian function I(x)
#
#
# Methods:
#
# Kernel.func(x) ... returns Kernel(x)
# 
#   Input: x - a numpy array of any dimension.
#
#   Output: the Kernel values at x. This is equal to Kernel(x) = A(x) + I(x) as written in the paper.
#
# Kernel.integral(N=1000) ... estimates the 2D integral of Kernel(x) in [-inf,inf] x [-inf,inf].
#                             it is not possible to do an integral in infinite domain, so the result is an approximation
#                             done in a domain large enough. It uses the trapezoid integral of scipy to integrate discretely.
# 
#   Input: N - mesh size of the grid. No particular need to change this value as N=1000 is enough for accuracy.
#
#   Output: the value of the (estimated) integral of Kernel(x) 
#
# Kernel.FFT() ... performs a fast fourier transform of Kernel(x) to show dominant frequencies.
#                  It sets the output (yf) as well as the frequency (xf) which will be used in the summary plot.
#
#
# Kernel.summary(x) ... returns the summary figure of the Kernel.
#
#   Input: x - the x-axis used for the plot. Only needed if the plot is drawn badly.
#
#   Output: a figure with the information below:
#           - plot of Kernel(x)
#           - FFT result of Kernel(x)
#           - (estimated) integral value of Kernel(x) with x in R^2
#

class Kernel():
  def __init__(self, _ampA, _ampI, _widthA, _widthI, _distA, _distI):
    self.A = Gaussian(_ampA, _widthA, _distA)
    self.I = Gaussian(_ampI, _widthI, _distI)

  def func(self,x):
    return self.A.func(x) + self.I.func(x)

  def integral(self, N=1000):
    end = np.max((12*self.A.width+self.A.dist,12*self.I.width+self.I.dist))
    X, Y = np.meshgrid(np.linspace(0,end,N),np.linspace(0,end,N))
    dx = end/(N-1)
    dist = (X**2 + Y**2)**0.5
    return 4*sp_int.trapezoid(sp_int.trapezoid(self.func(dist),dx=dx),dx=dx)

  def FFT(self):
    N = 10000
    lims = [0,N]
    x = np.linspace(lims[0],lims[1],N)
    yf = sp_fft.rfft(self.func(x))
    xf = sp_fft.rfftfreq(len(x))
    self.xf = xf
    self.yf = yf

  def summary(self,x=np.linspace(0,20,1000)):
    fig, ax = plt.subplots(1,2,figsize=(15,5))
    fig.suptitle('Kernel Summary')
    ax[0].fill_between(x,self.A.func(x),color='g',alpha=0.3,label=r'$A(x)$')
    ax[0].fill_between(x,self.I.func(x),color='r',alpha=0.3,label=r'$I(x)$')
    ax[0].plot(x,self.func(x),'k',label=r'$Kernel(x) = A(x) + I(x)$')
    ax[0].set_xlabel(r'Distance from signaling source $x$')
    ax[0].set_title('Kernel Shape (Integral value: {:.3f})'.format(self.integral()))
    ax[0].grid()
    ax[0].legend()
    self.FFT()
    ax[1].fill_between(self.xf,self.yf,color='k',alpha=0.8)
    ax[1].set_xticks([0,self.xf[-1]])
    ax[1].set_yticks([])
    ax[1].set_xlim(left=0,right=self.xf[-1])
    ax[1].set_xlabel('Frequencies')
    ax[1].set_title('FFT Profile') 

#
# class Kernel_Turing_model is the main model used to generate the Turing pattern.
# To solve the ODE defined in the paper, we iterate with time increment dt to numerically approximate the values at the grid.
#
# Parameters:
#
# Kernel_Turing_model.Kernel (class Kernel) ... the Kernel function Kernel(x)
#
# Kernel_Turing_model.X
# Kernel_Turing_model.Y ... x and y coordinates of each grid position. This can be generated from numpy function meshgrid().
#
# Kernel_Turing_model.dt ... the value of dt used to solve the ODE. Since our goal is to see the final steady state, it is not necessary to
#                            set this value too small (unless the scheme becomes unstable).
#
# Kernel_Turing_model.t ... the current time. This value is updated when the method .update_grid() is run
#
# Kernel_Turing_model.grid ... u(t,x,y) stored as a 2D matrix. This is updated according to the ODE using .update_grid()
#
#
# Methods:
#
# Kernel_Turing_model.stim() ... returns Stim(x,y) as explained in the paper. Uses 2D convolution (of scipy library) using periodic
#                                boundary conditions. The distance from origin (sqrt(x**2 + y**2)) is passed on to the argument of Kernel.
#                                Used in the method .update_grid() to calculate the value of u(t+dt,x,y) from u(t,x,y).
#
# Kernel_Turing_model.reset_grid() ... resets u(t,x,y) to u(0,x,y) with random samples from normal distribution. 

#
# Kernel_Turing_model.update_grid(deg=0.1) ... calculates u(t+dt,x,y) from u(t,x,y) using explicit scheme. The governing ODE is taken from
#                                              Equation (2) of the original paper. Since the value of "deg" was not specified in the paper,
#                                              here I set the default value to 0.1 (which can be changed as an argument).
#
# Kernel_Turing_model.gridplot() ... plots the image of u(t,x,y). If t is large enough and the Kernel is appropriate, then we should
#                                    observe a Turing pattern.
#

class Kernel_Turing_model():
  def __init__(self, _Kernel, _X, _Y, _dt):
    self.Kernel = _Kernel
    self.X = _X
    self.Y = _Y
    self.dt = _dt
    self.t = 0
    self.grid = np.random.randn(X.shape[0],X.shape[1])

  def stim(self):
    dist = (self.X**2 + self.Y**2)**0.5
    stim = sp_sgn.convolve2d(self.grid, self.Kernel.func(dist), mode='same', boundary='wrap')
    return np.clip(stim,0,1000)
  def reset_grid(self):
    self.grid = np.random.randn(X.shape[0],X.shape[1])
    self.t = 0

  def update_grid(self, deg=0):
    grid_new = self.grid + (self.stim() - deg*self.grid)*self.dt
    self.grid = grid_new
    self.t += self.dt

  def gridplot(self, ax):
    x = self.X[0,:]
    y = self.Y[:,0]
    ax.imshow(self.grid, cmap='gray', extent=(x[0],x[-1],y[0],y[-1]))
    ax.set_title('Result at t = {:.3f}'.format(self.t))


In [108]:
# Here is an example code: First we define our Kernel by passing the amplitude, width, and distance values the two Gaussians
# Keep in mind that the order of the arguments is (ampA, ampI, widthA, widthI, distA, distI)
K = Kernel(20.267,-2.133,1.817,5.835,0,0)
# Other examples:
# K = Kernel(17.192,-13.3333,1.18,1.18,8.3,10.7)
# K = Kernel(21.085,-19.733,0.739,0.935,10.3,8.7)
# K = Kernel(20.267,-2.133,1.817,5.835,0,0)

# Then we can see the summary of the Kernel using the method .summary()
K.summary()
# As explained in the paper, the FFT graph and the intergrated values are key factors which define the Turing patterns.

# Now we can construct our model by setting our domain and passing it with the Kernel to Kernel_Turing_model.
x = np.linspace(-100,100,201)
X, Y = np.meshgrid(x, x)
model = Kernel_Turing_model(K,X,Y,1.0)

# Then we use the update_grid() method to update the grid a several times: the number of iterations doesn't have to be too big
for i in range(10):
  print(i)
  model.update_grid()

# Finally we observe our Turing pattern
fix, ax = plt.subplots()
model.gridplot(ax)

0
1
2
3
4
5
6
7
8
9


In [109]:
# Code used for the graphics in report

fig, ax = plt.subplots(1,3,figsize=(15,5))
ampAs = [20.267,21.971,25.067]
for i in range(3):
  K = Kernel(ampAs[i],-2.133,1.817,5.835,0,0)
  model = Kernel_Turing_model(K,X,Y,1.0)
  for j in range(10):
    print((i,j))
    model.update_grid()
  model.gridplot(ax[i])
  ax[i].set_title('Integrated value = {:.3f}'.format(model.Kernel.integral()))

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)


In [23]:
# Here is an example code: First we define our Kernel by passing the amplitude, width, and distance values the two Gaussians
# Keep in mind that the order of the arguments is (ampA, ampI, widthA, widthI, distA, distI)
K = Kernel(20.267,-3.733,2.601,4.855,0,0)
# Other examples:
# K = Kernel(17.192,-13.3333,1.18,1.18,8.3,10.7)
# K = Kernel(21.085,-19.733,0.739,0.935,10.3,8.7)
# K = Kernel(20.267,-2.133,1.817,5.835,0,0)

# Then we can see the summary of the Kernel using the method .summary()
K.summary()
# As explained in the paper, the FFT graph and the intergrated values are key factors which define the Turing patterns.

# Now we can construct our model by setting our domain and passing it with the Kernel to Kernel_Turing_model.
x = np.linspace(-100,100,201)
X, Y = np.meshgrid(x, x)
model = Kernel_Turing_model(K,X,Y,1.0)

# Then we use the update_grid() method to update the grid a several times: the number of iterations doesn't have to be too big
for i in range(10):
  print(i)
  model.update_grid()

# Finally we observe our Turing pattern
fix, ax = plt.subplots()
model.gridplot(ax)

/Users/teratsujikanademe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/axes/_axes.py:5344: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
/Users/teratsujikanademe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py:1369: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


0
1
2
3
4
5
6
7
8
9


In [26]:
# Here is an example code: First we define our Kernel by passing the amplitude, width, and distance values the two Gaussians
# Keep in mind that the order of the arguments is (ampA, ampI, widthA, widthI, distA, distI)
K = Kernel(20.267,-2.133,1.817,5.835,0,0)
# Other examples:
# K = Kernel(17.192,-13.3333,1.18,1.18,8.3,10.7)
# K = Kernel(21.085,-19.733,0.739,0.935,10.3,8.7)
# K = Kernel(20.267,-2.133,1.817,5.835,0,0)

# Then we can see the summary of the Kernel using the method .summary()
K.summary()

/Users/teratsujikanademe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/axes/_axes.py:5344: ComplexWarning: Casting complex values to real discards the imaginary part
  pts[1:N+1, 1] = dep1slice
/Users/teratsujikanademe/opt/anaconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py:1369: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)
